<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/02.%20Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 2: End to End Machine Learning Project

Bab ini memandu pembaca melalui contoh proyek ML secara lengkap, dari awal hingga akhir, menggunakan dataset **California Housing Prices**. Tujuannya adalah untuk membangun model yang dapat memprediksi harga median rumah di sebuah distrik.

### 1. Mendapatkan Data

Langkah pertama adalah mengunduh dan memuat dataset. Kita akan membuat beberapa fungsi untuk mengotomatiskan proses ini.

```python
# Diperlukan untuk Google Colab
# Menyiapkan environment
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os
import tarfile
import urllib.request

# Lokasi download dan path penyimpanan data
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# Fungsi untuk mengunduh dan mengekstrak data
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

# Menjalankan fungsi download
fetch_housing_data()

# Fungsi untuk memuat data CSV ke dalam pandas DataFrame
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
```

### 2. Inspeksi Awal Data
Setelah data dimuat, kita perlu melihat struktur dan ringkasan statistiknya untuk mendapatkan pemahaman awal.

```python
# Menampilkan 5 baris pertama
print("Lima baris pertama:")
print(housing.head())
print("\n" + "="*50 + "\n")

# Menampilkan informasi ringkas (tipe data, nilai non-null)
print("Informasi dataset:")
housing.info()
print("\n" + "="*50 + "\n")

# Melihat ringkasan statistik fitur numerik
print("Deskripsi statistik:")
print(housing.describe())
print("\n" + "="*50 + "\n")

# Melihat distribusi data dengan histogram
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()
```

### 3. Membuat Test Set (Stratified Sampling)
Penting untuk menyisihkan *test set* sejak awal untuk mencegah *data snooping bias*. Kita akan menggunakan *stratified sampling* berdasarkan kategori pendapatan untuk memastikan *test set* representatif.

```python
# Membuat kategori pendapatan untuk stratified sampling
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Menghapus kolom income_cat karena hanya digunakan untuk splitting
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Membuat salinan data training untuk eksplorasi
housing = strat_train_set.copy()
```

### 4. Persiapan Data (Preprocessing Pipeline)
Tahap ini adalah yang paling krusial, di mana kita membersihkan dan mentransformasi data agar siap digunakan oleh model. Kita akan membangun sebuah *pipeline* untuk menangani fitur numerik dan kategorikal secara terpisah.

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Memisahkan fitur dari label
housing_labels = housing["median_house_value"].copy()
housing = housing.drop("median_house_value", axis=1)

# Mengambil nama kolom numerik dan kategorikal
housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

# Pipeline untuk fitur numerik: mengisi nilai kosong & menstandardisasi
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

# Pipeline penuh yang menggabungkan proses numerik dan kategorikal
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

# Menjalankan pipeline pada data training
housing_prepared = full_pipeline.fit_transform(housing)
```

### 5. Memilih dan Melatih Model
Kita akan melatih beberapa model dan mengevaluasinya menggunakan *cross-validation* untuk memilih yang terbaik.

```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Inisialisasi model
forest_reg = RandomForestRegressor(random_state=42)

# Evaluasi model menggunakan cross-validation
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

print("Hasil Evaluasi RandomForestRegressor:")
display_scores(rmse_scores)
```

### 6. Fine-Tuning Model
Setelah memilih model terbaik, kita mencari kombinasi *hyperparameter* optimal menggunakan `GridSearchCV`.

```python
from sklearn.model_selection import GridSearchCV

# Definisikan grid hyperparameter yang akan diuji
param_grid = [
    {'n_estimators': [10, 30, 50], 'max_features': [6, 8, 10]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)

# Menjalankan grid search
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

# Menampilkan hyperparameter terbaik
print("\nHyperparameter terbaik:", grid_search.best_params_)
```

### 7. Evaluasi Akhir pada Test Set
Terakhir, kita mengevaluasi model final pada *test set* yang telah kita sisihkan sejak awal.

```python
from sklearn.metrics import mean_squared_error

# Model terbaik dari Grid Search
final_model = grid_search.best_estimator_

# Memisahkan fitur dan label dari test set
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# Menjalankan pipeline pada test set (hanya transform, bukan fit_transform)
X_test_prepared = full_pipeline.transform(X_test)

# Membuat prediksi
final_predictions = final_model.predict(X_test_prepared)

# Menghitung RMSE final
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print("\nRMSE final pada test set:", final_rmse)
```
